# A fastai implementation for the famous MNIST handwritten digit dataset

In this knowledge contest, we are trying to predict the number of the handwritten digit.

**Version History**
* *Version 1* - Getting the notebook setup, importing basic modules, reading the dataset, building a simple model, evaluate results, no submission
* *Version 2* - Adding submisson, and callbacks
* *Version 3* - removed MixUp callback
* *Version 4* - cleaned up code, CrossEntropyLoss
* *Version 5* - removed training and testing images from output
* *Version 6* - Increased training epochs
* *Version 7* - Adding Mixed presision callbacks (Final submission?)
* *Version 8* - Editting and updating

## Background

I've tried to pull together a lot of the techniques I've used and seen other use.

I feel like if I ran this notebook enough times, I'd eventually find a model that would get perfect accuracy but obviously I'm just overfitting to score super high on the test set

### For beginners forking this notebook

I highly suggest for anyone interested in solving problems like this to look into the course by Sylvain Gugger and Jeremy Howard. They spent a lot of time building `fastai` into a great library for accessing the latest deep learning techniques.

[Practical Deep Learning for Coders](https://course.fast.ai/)

# Setup

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
%time
from fastai.vision.all import *
from fastprogress.fastprogress import progress_bar
import shutil

In [ ]:
INPUT_DIR = Path("../input/digit-recognizer")
INPUT_DIR.ls()
WORKING_DIR = Path("./")
TRAINING_DATA = WORKING_DIR / 'train'
TESTING_DATA = WORKING_DIR / 'test'

In [ ]:
train_df = pd.read_csv(INPUT_DIR / 'train.csv')
test_df = pd.read_csv(INPUT_DIR / 'test.csv')

In [ ]:
train_df.head()

# Convert CSV to images

In [ ]:
Y_train = train_df["label"]
X_train = train_df.drop("label", axis=1)

In [ ]:
X_train = X_train.values.reshape(-1, 28, 28)
X_test = test_df.values.reshape(-1, 28, 28)

In [ ]:
plt.imshow(X_train[0], cmap="gray")

# Move files into directory to make using fastai factory methods easier

In [ ]:
filenames = list(Y_train.index)
category = list(Y_train)

In [ ]:
for name, digit, image in progress_bar(list(zip(list(Y_train.index), category, X_train))):
    file_dir = TRAINING_DATA / str(digit)
    file_dir.mkdir(parents=True, exist_ok=True)
    file_name = f'{name}.png'
    file_path = file_dir / file_name
#     print(image.shape)
    plt.imsave(file_path, image)

In [ ]:
for name, image in progress_bar(list(zip(list(test_df.index), X_test))):
    file_dir = TESTING_DATA
    file_dir.mkdir(parents=True, exist_ok=True)
    file_name = f'{name}.png'
    file_path = file_dir / file_name
#     print(image.shape)
    plt.imsave(file_path, image)

# Model

In [ ]:
batch_tfms = [*aug_transforms(do_flip=False,), Normalize.from_stats(*imagenet_stats)]
metrics = [accuracy]
bs = 128

In [ ]:
dls = ImageDataLoaders.from_folder(TRAINING_DATA, valid_pct=0.2, batch_tfms=batch_tfms, bs=bs)
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=metrics, loss_func=CrossEntropyLossFlat())

In [ ]:
lrs = learn.lr_find()

In [ ]:
lrs

In [ ]:
cbs = [
    GradientAccumulation(),
    MixedPrecision(),
    SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=0.001),
    ReduceLROnPlateau(monitor='accuracy', comp=np.greater, min_delta=0.001, patience=2),
#     MixUp(0.4),
    EarlyStoppingCallback(monitor='accuracy', comp=np.greater, min_delta=0.001, patience=3),
      ]

In [ ]:
learn.fine_tune(15, lrs.lr_min, cbs=cbs)

# Evaluate Results

In [ ]:
%time
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
%time
interp.plot_top_losses(8, nrows=2)

# Submission

In [ ]:
SAMPLE_SUBMISSION = pd.read_csv(INPUT_DIR / 'sample_submission.csv')

In [ ]:
SAMPLE_SUBMISSION.head()

In [ ]:
test_images = list(Path(r'./test').glob('**/*'))
test_dl = dls.test_dl(test_images)

In [ ]:
preds = learn.get_preds(dl=test_dl, with_decoded=True)
preds = preds[2].tolist()

In [ ]:
ids = [int(img.stem) + 1 for img in test_images]

In [ ]:
submission = pd.DataFrame(ids, columns=['ImageId'])

In [ ]:
submission['Label'] = preds

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv",index=False)

# Remove Images

For some reason I can't make a submission from the notebook without deleting all of the saved training and test data files first.

In [ ]:
shutil.rmtree(TRAINING_DATA)
shutil.rmtree(TESTING_DATA)